In [56]:
# Importing packages and libraries:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm
import requests as req
import io

# Determinands:

In [57]:
# input determinands ids:
dtrs = input("Enter determinands ids (separated by space):").split()

Enter determinands ids (separated by space): 0111 0117 9686 0180 0348


In [58]:
# Dictionaries mapping each determinand to its label and abbreviation:
dtrs_labels = {
    "0111": "NH4 (#0111)",
    "0116": "NOx (#0116)",
    "0117": "NO3 (#0117)",
    "0118": "NO2 (#0118)",
    "9686": "Total N (#9686)",
    "0348": "Total P (#0348)",
    "0180": "PO4 (#0180)"
}

dtrs_abbrev = {
    "0111": "NH4",
    "0116": "NOx",
    "0117": "NO3",
    "0118": "NO2",
    "9686": "TN",
    "0348": "TP",
    "0180": "PO4"
}

# Study Period:

In [59]:
# input start and end date of the study period:
start_date_str = input("Enter start date (YYYY-MM-DD):")
end_date_str = input("Enter end date (YYYY-MM-DD):")

# Converting dates from strings into datetime objects:
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_yr = start_date.year
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_yr = end_date.year

Enter start date (YYYY-MM-DD): 2000-01-01
Enter end date (YYYY-MM-DD): 2024-12-31


# WQ Sites:

In [60]:
sites = input("Enter site ids (separated by space):").split()

Enter site ids (separated by space): AN-26M31 TH-PTHR0107 AN-51M01 AN-53M14 AN-NENE640D AN-WEN250 TH-PWAE0010 NE-43500692


# Fetching WQ Data:

In [61]:
# Creating nested dictionaries to store determinand & sites data:
dtrs_dic = {}
for dtr_id in dtrs:
    dtrs_dic[dtr_id] = {}


# WQA API Base url: 
base_url = "https://environment.data.gov.uk/water-quality"
## Request are made through the 'Observations' endpoint (site by site for each dtr).


# Iterating through predefined determinands:
for dtr in tqdm(dtrs,
                desc = f"Fetching concentration data for {len(dtrs)} determinands"
               ):
    
    dtr_dic = {}
    dtr_label = dtrs_labels.get(dtr)

    ## Defining the query parameters & headers (data type):
    params = {
        "dateFrom": start_date_str, ## start and end date parameters must be in string format here
        "dateTo": end_date_str,
        "limit": 250,
        "complianceOnly": "false", 
        "determinand": dtr
    }
    
    headers = {
        "accept": "text/csv",
        "Accept-Crs": "http://www.opengis.net/def/crs/EPSG/0/27700",
        "CSV-Header": "present",
        "API-Version": "1"
    }


    ## Looping through sites:
    for site_id in tqdm(
        sites,
        desc = f"Fetching {dtr_label} data across {len(sites)} sites"
    ):

        # The new api has a limit of 250 observation per query (site data will be retrieved in chunks):
        all_site_chunks = []
        skip = 0

        while True:
            ### Update pagination parameter "skip":
            params["skip"] = skip

            ### Sending the API request (observations 'with point notation' endpoint):
            obs_endpoint = f"{base_url}/sampling-point/{site_id}/observation"
            resp = req.get(obs_endpoint, params = params, headers = headers)

            ### 1) if the request did not work:
            if resp.status_code != 200:
                print(f"⛔ Failed Request Error {resp.status_code} for {dtr_label} at {site_id}")
                break

            ### 2) if request works, adding the site data to the dictionary:
            try:
                csv_text = resp.text.strip()
                if not csv_text or csv_text.count('\n') <= 1: # check for sites with exactly 250 rows!
                    break
                    
                site_chunk = pd.read_csv(
                io.StringIO(resp.text),
                dtype = {"determinand.notation": "str"}, # determinand id has leading zeroes on WIMS
                parse_dates = ["phenomenonTime"]
                )

                if site_chunk.empty:
                    break

                #### Cleaning of datetime columns, adding separate date and time columns:
                site_chunk['date'] = site_chunk['phenomenonTime'].dt.date
                site_chunk['time'] = site_chunk['phenomenonTime'].dt.time
                site_chunk['date'] = pd.to_datetime(site_chunk['date'])

                #### Seperating the result column into qual & result_num column using regex:
                pattern = r'^(<)?\s*([0-9.]+)$'
                site_chunk[['qual', 'result_num']] = site_chunk['result'].astype(str).str.extract(pattern)
                site_chunk['result_num'] = site_chunk['result_num'].astype(float)

                all_site_chunks.append(site_chunk)

                if len(site_chunk) < 250:
                    break
                skip += 250

            ### 3) In case of parsing errors:
            except pd.errors.ParserError:
                print(f"⚠️ Parser Error for {dtr_label} at {site_id}")
                break

            ### 4) Empty data error (no data for that site/dtr): 
            except pd.errors.EmptyDataError:
                print(f"⚠️ Empty Data Error for {dtr_label} at {site_id}")
                break
                    
        ### Merging site data:
        if all_site_chunks:
            site_data = pd.concat(all_site_chunks, ignore_index=True)
            dtr_dic[site_id] = site_data
        
            
    ## Assigning the created dtr dictionary to the main dtr dic:
    dtrs_dic[dtr] = dtr_dic                

Fetching concentration data for 5 determinands:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching NH4 (#0111) data across 8 sites:   0%|          | 0/8 [00:00<?, ?it/s]

Fetching NO3 (#0117) data across 8 sites:   0%|          | 0/8 [00:00<?, ?it/s]

⛔ Failed Request Error 204 for NO3 (#0117) at TH-PWAE0010


Fetching Total N (#9686) data across 8 sites:   0%|          | 0/8 [00:00<?, ?it/s]

⛔ Failed Request Error 204 for Total N (#9686) at TH-PWAE0010
⛔ Failed Request Error 204 for Total N (#9686) at NE-43500692


Fetching PO4 (#0180) data across 8 sites:   0%|          | 0/8 [00:00<?, ?it/s]

Fetching Total P (#0348) data across 8 sites:   0%|          | 0/8 [00:00<?, ?it/s]

In [64]:
# Melting all the dictionaries into one dataframe:
all_records = []

for dtr_id, dtr_dict in dtrs_dic.items():
    for site_id, site_df in dtr_dict.items():
        if isinstance(site_df, pd.DataFrame) and not site_df.empty:
            site_df = site_df.copy()
            all_records.append(site_df)

wqa = pd.concat(all_records, ignore_index=True)

wqa

,id,samplingPoint.notation,samplingPoint.prefLabel,samplingPoint.easting,samplingPoint.northing,samplingPoint.region,samplingPoint.area,samplingPoint.subArea,samplingPoint.samplingPointStatus,samplingPoint.samplingPointType,...,samplingPurpose,sampleMaterialType,determinand.notation,determinand.prefLabel,result,unit,date,time,qual,result_num
0,http://environment.data.gov.uk/water-quality/s...,AN-26M31,HUNDRED FOOT RIVER EARITH RD.BR.,539315,274729,Anglian,ANGLIAN - CAMBS AND BEDFORDSHIRE,ELY TEAM SUB AREA,OPEN,FRESHWATER - RIVERS,...,PLANNED INVESTIGATION (LOCAL MONITORING),RIVER / RUNNING SURFACE WATER,0111,Ammoniacal Nitrogen as N,<0.03,MILLIGRAM PER LITRE,2000-02-29,12:25:00,<,0.030
1,http://environment.data.gov.uk/water-quality/s...,AN-26M31,HUNDRED FOOT RIVER EARITH RD.BR.,539315,274729,Anglian,ANGLIAN - CAMBS AND BEDFORDSHIRE,ELY TEAM SUB AREA,OPEN,FRESHWATER - RIVERS,...,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,RIVER / RUNNING SURFACE WATER,0111,Ammoniacal Nitrogen as N,0.095,MILLIGRAM PER LITRE,2000-01-11,10:16:00,NaN,0.095
2,http://environment.data.gov.uk/water-quality/s...,AN-26M31,HUNDRED FOOT RIVER EARITH RD.BR.,539315,274729,Anglian,ANGLIAN - CAMBS AND BEDFORDSHIRE,ELY TEAM SUB AREA,OPEN,FRESHWATER - RIVERS,...,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,RIVER / RUNNING SURFACE WATER,0111,Ammoniacal Nitrogen as N,0.126,MILLIGRAM PER LITRE,2000-03-02,14:30:00,NaN,0.126
3,http://environment.data.gov.uk/water-quality/s...,AN-26M31,HUNDRED FOOT RIVER EARITH RD.BR.,539315,274729,Anglian,ANGLIAN - CAMBS AND BEDFORDSHIRE,ELY TEAM SUB AREA,OPEN,FRESHWATER - RIVERS,...,PLANNED INVESTIGATION (LOCAL MONITORING),RIVER / RUNNING SURFACE WATER,0111,Ammoniacal Nitrogen as N,0.042,MILLIGRAM PER LITRE,2000-02-29,17:10:00,NaN,0.042
4,http://environment.data.gov.uk/water-quality/s...,AN-26M31,HUNDRED FOOT RIVER EARITH RD.BR.,539315,274729,Anglian,ANGLIAN - CAMBS AND BEDFORDSHIRE,ELY TEAM SUB AREA,OPEN,FRESHWATER - RIVERS,...,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,RIVER / RUNNING SURFACE WATER,0111,Ammoniacal Nitrogen as N,0.046,MILLIGRAM PER LITRE,2000-05-10,12:00:00,NaN,0.046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15552,http://environment.data.gov.uk/water-quality/s...,NE-43500692,BIRTLEY STW FINAL EFFLUENT (TO LAMESLEY WETLAN...,425630,556730,NorthEast,NORTH EAST - NORTHUMBERLAND DURHAM AND TEES,NORTHUMBERLAND & TYNE,OPEN,SEWAGE DISCHARGES - FINAL/TREATED EFFLUENT - W...,...,COMPLIANCE AUDIT (PERMIT),FINAL SEWAGE EFFLUENT,0348,"Phosphorus, Total as P",2.73,MILLIGRAM PER LITRE,2008-10-28,13:10:00,NaN,2.730
15553,http://environment.data.gov.uk/water-quality/s...,NE-43500692,BIRTLEY STW FINAL EFFLUENT (TO LAMESLEY WETLAN...,425630,556730,NorthEast,NORTH EAST - NORTHUMBERLAND DURHAM AND TEES,NORTHUMBERLAND & TYNE,OPEN,SEWAGE DISCHARGES - FINAL/TREATED EFFLUENT - W...,...,COMPLIANCE AUDIT (PERMIT),FINAL SEWAGE EFFLUENT,0348,"Phosphorus, Total as P",2.65,MILLIGRAM PER LITRE,2008-11-14,10:45:00,NaN,2.650
15554,http://environment.data.gov.uk/water-quality/s...,NE-43500692,BIRTLEY STW FINAL EFFLUENT (TO LAMESLEY WETLAN...,425630,556730,NorthEast,NORTH EAST - NORTHUMBERLAND DURHAM AND TEES,NORTHUMBERLAND & TYNE,OPEN,SEWAGE DISCHARGES - FINAL/TREATED EFFLUENT - W...,...,COMPLIANCE AUDIT (PERMIT),FINAL SEWAGE EFFLUENT,0348,"Phosphorus, Total as P",2.7,MILLIGRAM PER LITRE,2008-12-11,10:15:00,NaN,2.700
15555,http://environment.data.gov.uk/water-quality/s...,NE-43500692,BIRTLEY STW FINAL EFFLUENT (TO LAMESLEY WETLAN...,425630,556730,NorthEast,NORTH EAST - NORTHUMBERLAND DURHAM AND TEES,NORTHUMBERLAND & TYNE,OPEN,SEWAGE DISCHARGES - FINAL/TREATED EFFLUENT - W...,...,COMPLIANCE AUDIT (PERMIT),FINAL SEWAGE EFFLUENT,0348,"Phosphorus, Total as P",3.2,MILLIGRAM PER LITRE,2009-01-28,11:43:00,NaN,3.200
